Visualization #3

    Is the distribution of students in NCWIT data set similar to the population for ACT scores?

In [2]:
import pandas as pd
import bokeh
import numpy as np
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999

In [113]:
ncwit_df = pd.read_csv("NCWIT-Data Cleaned.csv", low_memory = False)
# Left this out so code below only calculates reported values.
#ncwit_df = ncwit_df.fillna(0)
ncwit_df.head(30)

,Record#,CIP# Only,Major Program Name,Degree Level,NCWIT Participant,Institution,What degrees does your institution offer?,School Year,When do students typically declare their major?,"Enroll, Female: Avg. ACT Math Score (Enrl F)","Enroll, Female: Asian (Enrl F)","Enroll, Female: Average Age (Enrl F)","Enroll, Female: Average HS GPA (Enrl F)","Enroll, Female: Avg. SAT Math Score (Enrl F)","Enroll, Female: Black/African American (Enrl F)","Enroll, Female: Hispanics of any race (Enrl F)","Enroll, Female: American Indian/Alaska Native (Enrl F)","Enroll, Female: Native Hawaiian/Other Pacific Islander (Enrl F)","Enroll, Female: Two or more races (Enrl F)","Enroll, Female: New Enrollments (Enrl F)","Enroll, Female: Transfer Students (Enrl F)","Enroll, Female: US Citizens (Enrl F)","Enroll, Female: White (Enrl F)","Enroll, Male: Avg. ACT Math Score (Enrl M)","Enroll, Male: Asian (Enrl M)","Enroll, Male: Average Age (Enrl M)","Enroll, Male: Average HS GPA (Enrl M)","Enroll, Male: Avg. SAT Math Score (Enrl M)","Enroll, Male: Black/African American (Enrl M)","Enroll, Male: Hispanics of any race (Enrl M)","Enroll, Male: American Indian/Alaska Native (Enrl M)","Enroll, Male: Native Hawaiian/Other Pacific Islander (Enrl M)","Enroll, Male: Two or more races (Enrl M)","Enroll, Male: New Enrollments (Enrl M)","Enroll, Male: Transfer Students (Enrl M)","Enroll, Male: US Citizens (Enrl M)","Enroll, Male: White (Enrl M)","Freshmen, Female: Asian (Fshm F)","Freshmen, Female: Black/African American (Fshm F)","Freshmen, Female: Cumulative GPA (Fshm F)","Freshmen, Female: Enrolled in DIFFERENT MAJOR (Fshm F)","Freshmen, Female: Enrolled in SAME MAJOR (Fshm F)","Freshmen, Female: Graduated (Fshm F)","Freshmen, Female: Hispanics of any race (Fshm F)","Freshmen, Female: Left Institution (not graduated) (Fshm F)","Freshmen, Female: American Indian/Alaska Native (Fshm F)","Freshmen, Female: Native Hawaiian/Other Pacific Islander (Fshm F)","Freshmen, Female: Two or more races (Fshm F)","Freshmen, Female: Total Declared Majors (Fshm F)","Freshmen, Female: US Citizens (Fshm F)","Freshmen, Female: White (Fshm F)","Freshmen, Male: Asian (Fshm M)","Freshmen, Male: Black/African American (Fshm M)","Freshmen, Male: Cumulative GPA (Fshm M)","Freshmen, Male: Enrolled in DIFFERENT MAJOR (Fshm M)","Freshmen, Male: Enrolled in SAME MAJOR (Fshm M)","Freshmen, Male: Graduated (Fshm M)","Freshmen, Male: Hispanics of any race (Fshm M)","Freshmen, Male: Left Institution (not graduated) (Fshm M)","Freshmen, Male: American Indian/Alaska Native (Fshm M)","Freshmen, Male: Native Hawaiian/Other Pacific Islander (Fshm M)","Freshmen, Male: Two or more races (Fshm M)","Freshmen, Male: Total Declared Majors (Fshm M)","Freshmen, Male: US Citizens (Fshm M)","Freshmen, Male: White (Fshm M)","Sophomores, Female: Asian (Sph. F)","Sophomores, Female: Black/African American (Sph. F)","Sophomores, Female: Cumulative GPA (Sph. F)","Sophomores, Female: Enrolled in DIFFERENT MAJOR (Sph. F)","Sophomores, Female: Enrolled in SAME MAJOR (Sph. F)","Sophomores, Female: Graduated (Sph. F)","Sophomores, Female: Hispanics of any race (Sph. F)","Sophomores, Female: Left Institution (not graduated) (Sph. F)","Sophomores, Female: American Indian/Alaska Native (Sph. F)","Sophomores, Female: Native Hawaiian/Other Pacific Islander (Sph. F)","Sophomores, Female: Two or more races (Sph. F)","Sophomores, Female: Total Declared Majors (Sph. F)","Sophomores, Female: US Citizens (Sph. F)","Sophomores, Female: White (Sph. F)","Sophomores, Male: Asian (Sph. M)","Sophomores, Male: Black/African American (Sph. M)","Sophomores, Male: Cumulative GPA (Sph. M)","Sophomores, Male: Enrolled in DIFFERENT MAJOR (Sph. M)","Sophomores, Male: Enrolled in SAME MAJOR (Sph. M)","Sophomores, Male: Graduated (Sph. M)","Sophomores, Male: Hispanics of any race (Sph. M)","Sophomores, Male: Left Institution (not graduated) (Sph. M)","Sophomores, Male: American Indian/Alaska Native (Sph. M)","Sophomores, Male: Native Hawaiian/Other Paci

In [109]:
ACT_df = pd.read_csv("ACT Data.csv")
#print(ACT_df)
ncwit2016 = ncwit_df.loc[ncwit_df["School Year"] == "2016-2017"].copy()
#ncwit2016.head()


In [112]:
# https://stackoverflow.com/questions/12207326/frequency-table-for-a-single-variable
# Code finds the # of unique values for ACT scores (after rounding and dropping nan values) and
# returns the count of each observation of each value
fVal, fCounts = np.unique(ncwit2016["Enroll, Female: Avg. ACT Math Score (Enrl F)"].round(0).dropna(), return_counts=True)

fCounts = fCounts/np.sum(fCounts)
print(fVal,fCounts)

mVal, mCounts = np.unique(ncwit2016["Enroll, Male: Avg. ACT Math Score (Enrl M)"].round(0).dropna(), return_counts=True)
mCounts = mCounts/np.sum(mCounts)
print(mVal,mCounts)

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from scipy.stats import norm

output_notebook()
# New figure for Female data.
p = figure(x_range= [0,36.5], plot_height = 500, title = "2016 Average Score Distribution - Female")
p.vbar(x=fVal, top = fCounts, width = 0.9)
p.y_range.start = 0
# Generate a line for the distribution of scores that year for Females
x = np.linspace(0,36,500)
p.line(x, norm.pdf(x, 21, 5.5), line_width=2, color = "red")

# New figure for Male data.
q = figure(x_range= [0,36.5], plot_height = 500, title = "2016 Average Score Distribution - Male")
q.vbar(x=mVal, top = mCounts, width = 0.9)
q.y_range.start = 0
# Generate a line for the distribution of scores that year for Males
q.line(x, norm.pdf(x, 21.1, 5.5), line_width=2, color = "red")

grid = gridplot([p], [q])

show(grid)

[21. 27. 28. 29. 30. 31. 32. 33. 34.] [0.12 0.12 0.08 0.12 0.16 0.16 0.12 0.08 0.04]
[20. 24. 27. 28. 29. 30. 31. 32. 33. 34.] [0.11538462 0.03846154 0.03846154 0.19230769 0.07692308 0.07692308
 0.11538462 0.23076923 0.03846154 0.07692308]


Loading BokehJS ...